In [1]:
import pandas as pd
import numpy as np
import re
import IPython.display
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Model
from keras.layers import Input, Dense, Dropout, LSTM, Flatten
from keras.utils import to_categorical
from keras.utils import plot_model
from nltk.tokenize import word_tokenize
import spacy
nlp = spacy.load('en', vectors='en_glove_cc_300_1m_vectors')

Using TensorFlow backend.


In [2]:
headers = ['song_id', 'translated_names']
songs = pd.read_csv('../New_Data/tr_songs.csv', usecols = headers)
duplicated_idx = songs.duplicated(subset='song_id', keep='first')
songs = songs[~duplicated_idx]
songs['song_name'] = songs['translated_names'].map(str).apply(lambda x : ''.join([i for i in re.findall(r'[a-zA-Z_\s]', x)]))
songs['song_name'] = songs['song_name'].map(str).apply(lambda x : re.sub(r'\s+',' ',x))

In [3]:
#index 9 was missing so had to reindex
songs = songs.reset_index()

In [4]:
def generate_songs_tensor(song_names, nlp, steps):
    assert not isinstance(song_names, basestring)
    nb_samples = len(song_names)
    word_vec_dim = nlp(song_names[0].decode('utf8'))[0].vector.shape[0]
    song_tensor = np.zeros((nb_samples, steps, word_vec_dim))
    for i in xrange(len(song_names)):
        tokens = nlp(song_names[i].decode('utf8'))
        for j in xrange(len(tokens)):
            if j<steps:
                song_tensor[i,j,:] = tokens[j].vector

    return song_tensor

In [5]:
seq_length = 25
X = generate_songs_tensor(songs['song_name'], nlp, seq_length)

In [6]:
songs_mapper = dict()
songs_mapper = dict(zip(songs['song_id'], X))

In [ ]:
import cPickle as pickle
with open('../New_Data/LSTM_song_embeddings/song_mapper.pkl', 'wb') as fw:
    pickle.dump(songs_mapper, fw)